In [1]:
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
import sklearn.utils
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import nltk
import random
import re
from nltk.stem import PorterStemmer
import pandas as pd
import numpy as np

#nltk.download('punkt')
#nltk.download('stopwords')

In [2]:
tweets = pd.read_csv("tweets.csv", encoding='latin1', usecols=[0, 5], header=None, delimiter=',', quotechar='"')
tweets.columns = ['lable', 'text']
print(tweets.info())
tweets = sklearn.utils.shuffle(tweets, random_state=0)
tweets = tweets[:10000]
z = 0
o = 0
for n in tweets.lable:
    if(n == 0):
        z += 1
    else:
        o +=1

print("negative: ",z, "\t","Positive: ", o)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 2 columns):
lable    1600000 non-null int64
text     1600000 non-null object
dtypes: int64(1), object(1)
memory usage: 24.4+ MB
None
negative:  4954 	 Positive:  5046


In [15]:
def prepare(data):
    X = []
    sent_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
    stopwords = nltk.corpus.stopwords.words('english')
    whitelist = ["n't", "not", "no"]
    Stemmer = PorterStemmer()
    for row in data.text:
        row = re.sub(r'^https?:\/\/.*[\r\n]*', '', row, flags = re.MULTILINE) # Removing URLs
        row = re.sub(r'@\w+', '', row, flags = re.MULTILINE) # Removing accounts tag(@Ali)
        row = re.sub(r'[^\w\s]','', row) # Removing Punctuation
        row = sent_tokenizer.tokenize(row)
        tokens = []
        for sentence in row:
            words = nltk.word_tokenize(sentence)
            # Removing stopwords, Stemming and Converting every token to lowercase
            tokens += [Stemmer.stem(token.lower()) for token in words if (token.lower() not in stopwords or token in whitelist) and len(token) > 1] 
        row = tokens
        X += [' '.join(row)]
        
    #print(X[:10])
    return X

In [16]:
def Extract_Features(tweets):
    #print(tweets[:10])
    vectorizer = TfidfVectorizer()
    
    features = vectorizer.fit_transform(tweets)
    return features

In [17]:
X = Extract_Features(prepare(tweets))
y = tweets.lable
X_train, X_test, y_train, y_test  = train_test_split(X, y, test_size=0.20, random_state=47)

In [18]:
model = LogisticRegressionCV(cv=5, verbose=1, solver='newton-cg')
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    9.3s finished


In [19]:
print(classification_report(y_test, y_pred, target_names=['Negative', 'Positive']))
print("Logistic Regression Accuracy =", accuracy_score(y_test, y_pred) * 100, "%")

              precision    recall  f1-score   support

    Negative       0.78      0.71      0.75      1004
    Positive       0.74      0.80      0.77       996

   micro avg       0.76      0.76      0.76      2000
   macro avg       0.76      0.76      0.76      2000
weighted avg       0.76      0.76      0.76      2000

Logistic Regression Accuracy = 75.7 %
